<a href="https://colab.research.google.com/github/rudycav/Equities-Digital-Assets-Prediction-Model/blob/main/Project/%20Ethereum_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import numpy as np
import math
import tensorflow
import requests
import pandas as pd
import matplotlib.pylab as plt
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler

In [172]:
def get_crypto_price(symbol, exchange, start_date = None):
    api_key = ' E7KN41UXPEIKWXIK'
    api_url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={exchange}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    df = pd.DataFrame(raw_df['Time Series (Digital Currency Daily)']).T
    df = df.rename(columns = {'1a. open (USD)': 'open', '2a. high (USD)': 'high', '3a. low (USD)': 'low', '4a. close (USD)': 'close', '5. volume': 'volume'})

    for i in df.columns:
        df[i] = df[i].astype(float)
    df.index = pd.to_datetime(df.index)
    df = df.iloc[::-1].drop(['1b. open (USD)', '2b. high (USD)', '3b. low (USD)', '4b. close (USD)', '6. market cap (USD)', 'open', 'high', 'low', 'volume'], axis = 1)

    if start_date:
        df = df[df.index >= start_date]
    return df

eth_df = get_crypto_price(symbol = 'ETH', exchange = 'USD', start_date = '2016-01-01')
eth_df.head()

,close
2019-10-16,174.47
2019-10-17,177.16
2019-10-18,172.74
2019-10-19,171.79
2019-10-20,175.22


In [173]:
eth_df.shape

(1000, 1)

In [174]:
dataset_length = math.ceil(len(eth_df) * 0.7)


In [175]:
def scale_data(scaler):
  scaler = MinMaxScaler(feature_range=(0,1))
  scaled_data = scaler.fit_transform(eth_df)

  return scaled_data, scaler

scaled_data, scaler = scale_data(eth_df)


In [176]:
training_data = scaled_data[0: dataset_length, :]
testing_data = scaled_data[dataset_length -30: ,:]

In [177]:
def training_set(training_data):
  
  x_train, y_train = [], []

  for days in range(30, len(training_data)):
    x_train.append(training_data[days-30: days, 0])
    y_train.append(training_data[days,0])

  x_train, y_train = np.array(x_train), np.array(y_train)
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

  return x_train, y_train

x_train, y_train = training_set(training_data)


In [178]:
x_train.shape

(670, 30, 1)

In [179]:
def neural_network(x_train, y_train, epochs, batch_size):

  model = Sequential()

  model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
  model.add(Dropout(0.2))
  model.add(LSTM(units=50, return_sequences=True))
  model.add(Dropout(0.2))
  model.add(LSTM(units=50))
  model.add(Dropout(0.2))
  model.add(Dense(units=1))

  model.compile(optimizer='adam', loss='mean_squared_error')
  model.fit(x_train, y_train, epochs, batch_size)
 
  return model

neural_network = neural_network(x_train, y_train, epochs=25, batch_size=32)

Epoch 1/32
27/27 [==============================] - 8s 44ms/step - loss: 0.0185
Epoch 2/32
27/27 [==============================] - 1s 47ms/step - loss: 0.0043
Epoch 3/32
27/27 [==============================] - 1s 54ms/step - loss: 0.0040
Epoch 4/32
27/27 [==============================] - 1s 46ms/step - loss: 0.0037
Epoch 5/32
27/27 [==============================] - 1s 51ms/step - loss: 0.0032
Epoch 6/32
27/27 [==============================] - 1s 54ms/step - loss: 0.0030
Epoch 7/32
27/27 [==============================] - 1s 46ms/step - loss: 0.0030
Epoch 8/32
27/27 [==============================] - 1s 51ms/step - loss: 0.0032
Epoch 9/32
27/27 [==============================] - 1s 45ms/step - loss: 0.0033
Epoch 10/32
27/27 [==============================] - 1s 44ms/step - loss: 0.0032
Epoch 11/32
27/27 [==============================] - 1s 51ms/step - loss: 0.0024
Epoch 12/32
27/27 [==============================] - 1s 51ms/step - loss: 0.0026
Epoch 13/32
27/27 [==================

In [182]:
def testing_set(testing_data):
  
  x_test = []
  y_test = eth_df.values[dataset_length:, :]

  for days in range(30, len(testing_data)):
    x_test.append(testing_data[days-30: days, 0])
 
  x_test = np.array(x_test)
  x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

  return x_test, y_test

x_test, y_test = testing_set(testing_data)

In [184]:
predict = model.predict(x_test)
predict = scaler.inverse_transform(predict)


In [185]:
rmse =np.sqrt(np.mean(((predict- y_test)**2)))
rmse

306.5694465867666